# Data Preprocessing

## Note on Onehot Encoding

To prevent cluttering the database with several extra columns (and thus making it harder to maintain) the data will not be onehot encoded at this point. Instead onehot encoding will be applied to the dataset on demand by the ML service when requested.

This applies to the following columns:
 - AccidentType
 - Dominant injury
 - Vehicle Type
 - Weather Conditions
 - Accident Description
 - Injury Description
 - Gender

## Initial Data

In [ ]:
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

In [ ]:
claimData = pd.read_csv('OriginalClaimData.csv')

In [ ]:
pd.set_option('display.max_columns', None)  
print(claimData.columns)
print(claimData)

In [ ]:
print(f"Dataset shape: {claimData.shape}")
claimData.info()
claimData.describe().T


In [ ]:
missing_values = claimData.isnull().sum()
print("\nMissing values per column:")
print(missing_values[missing_values > 0])

In [ ]:

# Identify categorical and numerical columns
categorical_cols = claimData.select_dtypes(include=['object', 'string']).columns.tolist()
numerical_cols = claimData.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove the target variable from the feature lists
target_col = 'SettlementValue'
if target_col in numerical_cols:
    numerical_cols.remove(target_col)

print(f"\nCategorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")
print(f"Target column: {target_col}")


## Handling Missing Data

### Removing Missing Values

Since the settlement value is what's being predicted, any row with a null settlement value is invalid

In [ ]:
removableValueColumns = []
removableValueColumns.append('SettlementValue')

All values in injury prognosis have an associated letter code, since there is no provided letter code for missing/0 month prognoses there is no valid default value so any data rows with a missing prognosis is removed.

In [ ]:
removableValueColumns.append('Injury_Prognosis')

Dates and ages dont have good default values so they're removed. For number of passengers, while 1 seems a sensible default, this isn't necessarily a valid inference and is therefore removed.

In [ ]:
removableValueColumns.append('Accident Date')
removableValueColumns.append('Claim Date')
removableValueColumns.append('Number of Passengers')
removableValueColumns.append('Driver Age')
removableValueColumns.append('Vehicle Age')

In [ ]:
for rowIndex, rowData in claimData.iterrows():
    for col in removableValueColumns:
        if pd.isna(rowData[col]):
            claimData.drop(index=rowIndex, inplace=True)
            break

In [ ]:
claimData.reset_index(drop=True, inplace=True)    

print(claimData.isnull().any(axis = 1).sum())
print(len(claimData))

### Replace Missing Values

The following columns all represent monetary values. A missing value is interpreted as no value and thus they are defaulted to 0.

In [ ]:
defaultZeroColumns = ['SpecialHealthExpenses', 'SpecialReduction', 'SpecialOverage',
                      'GeneralRest', 'SpecialAdditionalInjury', 'SpecialEarningsLoss',
                      'SpecialUsageLoss', 'SpecialMedications', 'SpecialAssetDamage',
                      'SpecialRehabilitation', 'SpecialFixes', 'GeneralFixed',
                      'GeneralUplift', 'SpecialLoanerVehicle', 'SpecialTripCosts',
                      'SpecialJourneyExpenses', 'SpecialTherapy']

for rowIndex, rowData in claimData.iterrows():
    for col in defaultZeroColumns:
        if pd.isna(rowData[col]):
            claimData.at[rowIndex, col] = 0
            
print(claimData.isnull().any(axis = 1).sum())

The following columns are all boolean values. A missing value here is interpreted as false.

In [ ]:
defaultNoColumns = ['Exceptional_Circumstances', 'Minor_Psychological_Injury', 'Whiplash', 'Police Report Filed', 'Witness Present']

for rowIndex, rowData in claimData.iterrows():
    for col in defaultNoColumns:
        if pd.isna(rowData[col]):
            claimData.at[rowIndex, col] = "No"

In [ ]:
print(claimData.isnull().any(axis = 1).sum())

### Remaining Missing Values

Any missing values that haven't been handled so far are in columns that will later be one hot encoded at which point the missing value will no longer be an issue as it will instead be represented by a 0 across all relevant onehot columns.

## Duplicate Values

In [ ]:
# No duplicated rows
duplicates = claimData[claimData.duplicated(keep=False)]  # Finds all duplicates
# print(duplicates)

settlementCount = claimData['SettlementValue'].value_counts()
print(settlementCount.head(10))

£520 appears nearly twice as much as other most frequent values. We could maybe look at removing half these sample and how this affects our model.

## Data Visualization

In our dataset we have two protected characteristics:
- Age
- Gender

We can plot these columns to visualise how the data is distributed and if it is representative/ balanced.

In [ ]:
sns.displot(claimData['Gender'].to_numpy())
sns.displot(data=claimData, x="Driver Age", col="Gender", kde=True, col_wrap=4)

We have a good balance of sample size across gender. With only slightly more male than other categories.

We also have a good distribution of age representation within these groups.

In [ ]:
sns.displot(data=claimData, x="SettlementValue", col="Gender", kde=True, col_wrap=4)

Distribution of settlement value seems similar across all genders.

In [ ]:
def age_group(age):
    if age <= 30:
        return "Under 30"
    if age <= 40:
        return "Under 40"
    if age <= 50:
        return "Under 50"
    if age <= 60:
        return "Under 60"
    if age <= 70:
        return "Under 70"
    if age <= 80:
        return "Under 80"
    
age_df = claimData[["Driver Age", "SettlementValue"]].copy()

age_df["AgeGroup"] = age_df['Driver Age'].apply(age_group)

sns.displot(data=age_df, x="SettlementValue", col="AgeGroup", kde=True, col_wrap=4)
        

In [ ]:
# Check for biases in settlement values by protected characteristics
if 'Gender' in claimData.columns:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Gender', y=target_col, data=claimData)
    plt.title('Settlement Values by Gender')
    plt.tight_layout()
    plt.show()

In [ ]:
if 'DriverAge' in claimData.columns:
    # Create age groups for visualization
    claimData['AgeGroup'] = pd.cut(claimData['DriverAge'], 
                                    bins=[0, 30, 40, 50, 60, 70, 80],
                                    labels=['Under 30', '30-40', '40-50', '50-60', '60-70', '70-80'])
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='AgeGroup', y=target_col, data=claimData)
    plt.title('Settlement Values by Age Group')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


Above plots show settlement value by age groups. All have a very similar distribution.

Based on the above plots our dataset is balanced and representative in terms of protected characteristics. We can look at removing gender if it's not needed as it shouldn't have any bearing on our problem.

## Reducing Accuracy

Accident date and claim date are both recorded down to the millisecond which is far more accurate than is reasonably possible or necessary. Furthermore, since the time value for is identical across both fields it is likely that it was added automatically and/or by mistake.

In [ ]:
for rowIndex, rowData in claimData.iterrows():
    # The date portion of the data is always the first 10 characters
    accidentDateOnly = rowData["Accident Date"][:10]
    claimDateOnly = rowData["Claim Date"][:10]
    
    claimData.at[rowIndex, "Accident Date"] = accidentDateOnly
    claimData.at[rowIndex, "Claim Date"] = claimDateOnly
    
print(claimData[["Accident Date", "Claim Date"]])

## Data Types

Currently the injury prognosis is stored as a string in the format "X months" and preceded by a letter code. The letter code and "months" text is completely unnecessary and thus are being removed and the field converted to an integer representing the prognosis in months.

In [ ]:
i = 0
for cellData in claimData["Injury_Prognosis"]:
    months = int(''.join(c for c in cellData if c.isdigit()))
    claimData.at[i, "Injury_Prognosis"] = months
    i += 1
    
print(claimData["Injury_Prognosis"])

There are several columns which only contain "Yes" or "No" and as such are better represented boolean or binary values.

In [ ]:
binaryCols = defaultNoColumns

for col in binaryCols:
    i = 0
    for cellData in claimData[col]:
        val = 1 if cellData == "Yes" else 0
        claimData.at[i, col] = val
        i += 1
        
print(claimData[binaryCols])

In order for an AI to properly train on and understand a date, we need to convert it to a numerical value first, such as unix time or in our case (since we are using date only) julian date.

1. First converts the string dates to Julian format
2. Fixes the bug by using claimDate.year 
3. Calculates the difference to get days between dates

In [ ]:
# for rowIndex, rowData in claimData.iterrows():
#     accidentDate = rowData["Accident Date"]
#     claimDate = rowData["Claim Date"]
    
#     accidentDate = datetime.datetime.strptime(accidentDate, '%Y-%m-%d')
#     accidentJulianDay = accidentDate.strftime('%j')
#     accidentJulianDate = int(f"{accidentDate.year}{accidentJulianDay}")
    
#     claimDate = datetime.datetime.strptime(claimDate, '%Y-%m-%d')
#     claimJulianDay = claimDate.strftime('%j')
#     claimJulianDate = int(f"{accidentDate.year}{claimJulianDay}")
    
#     claimData.at[rowIndex, "Accident Date"] = accidentJulianDate
#     claimData.at[rowIndex, "Claim Date"] = claimJulianDate
    
# print(claimData[["Accident Date", "Claim Date"]])

# Calculate days between accident and claim dates

for rowIndex, rowData in claimData.iterrows():
    accidentDate = rowData["Accident Date"]
    claimDate = rowData["Claim Date"]
    
    accidentDate = datetime.datetime.strptime(accidentDate, '%Y-%m-%d')
    accidentJulianDay = accidentDate.strftime('%j')
    accidentJulianDate = int(f"{accidentDate.year}{accidentJulianDay}")
    
    claimDate = datetime.datetime.strptime(claimDate, '%Y-%m-%d')
    claimJulianDay = claimDate.strftime('%j')

    claimJulianDate = int(f"{claimDate.year}{claimJulianDay}")
    
    claimData.at[rowIndex, "Accident Date"] = accidentJulianDate
    claimData.at[rowIndex, "Claim Date"] = claimJulianDate

claimData['DaysBetweenAccidentAndClaim'] = claimData["Claim Date"] - claimData["Accident Date"]


In [ ]:
claimData.convert_dtypes().dtypes

## Skewness

Checking for possible skewness in the dataset. This plots how often the settlement values were awarded. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(claimData[target_col], kde=True)
plt.title('Distribution of Settlement Values')
plt.xlabel('Settlement Value')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
skewness = claimData[target_col].skew()
print(f"Skewness of target variable: {skewness}")

With the skewness being above 1, log transformation can be used to make it more even. This would mean the data will need to be transformed before inferencing. 

## Correlation Matrix

In [ ]:
if 'DriverAge' in claimData.columns:
    # Create age groups for visualization
    claimData['AgeGroup'] = pd.cut(claimData['DriverAge'], 
                                    bins=[0, 30, 40, 50, 60, 70, 80],
                                    labels=['Under 30', '30-40', '40-50', '50-60', '60-70', '70-80'])
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='AgeGroup', y=target_col, data=claimData)
    plt.title('Settlement Values by Age Group')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
corr_matrix = claimData.corr(numeric_only=True)
plt.figure(figsize=(12, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=1)
plt.title("Feature Correlation with Target")
plt.show()

In [ ]:
# top correlations with target
target_correlations = corr_matrix[target_col].sort_values(ascending=False)
print("\nTop 10 features correlated with settlement value:")
print(target_correlations[:10])


## Column Names

Column names in the dataset use varying naming conventions including PascalCase, Snake_Case and some are simply written with spaces. The database being used has all column names in PascalCase so for clarity and consistency all columns in the dataset will be converted. 

In [ ]:
columnRenameDict = {
    "Injury_Prognosis":"InjuryPrognosis",
    "Exceptional_Circumstances":"ExceptionalCircumstances",
    "Minor_Psychological_Injury":"MinorPsychologicalInjury",
    "Dominant injury":"DominantInjury",
    "Vehicle Type":"VehicleType",
    "Weather Conditions":"WeatherConditions",
    "Accident Date":"AccidentDate",
    "Claim Date":"ClaimDate",
    "Vehicle Age":"VehicleAge",
    "Driver Age":"DriverAge",
    "Number of Passengers":"NumberOfPassengers",
    "Accident Description":"AccidentDescription",
    "Injury Description":"InjuryDescription",
    "Police Report Filed":"PoliceReportFiled",
    "Witness Present":"WitnessPresent"
}

claimData = claimData.rename(columns=columnRenameDict)
print(claimData.columns)

In [ ]:
claimData.to_csv('ProcessedClaimData.csv', index=False)